# Question 1

In [1522]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [1523]:
# Create the optimization model
model = gb.Model("Question 1: Canola Oil")

In [1524]:
# Read costs from CSV files
direct_shipping_costs_df = pd.read_csv(r"C:\Users\gabri\Downloads\Cost_Production_to_Refinement.csv")
shipping_to_transsipment_costs_df = pd.read_csv(r"C:\Users\gabri\Downloads\Cost_Production_to_Transshipment.csv")
transsipment_to_refinement_costs_df = pd.read_csv(r"C:\Users\gabri\Downloads\Cost_Transshipment_to_Refinement.csv")

In [1525]:
# Extract the "Cost" column
direct_shipping_costs = direct_shipping_costs_df['Cost'].values.reshape(25, -1)
shipping_to_transsipment_costs = shipping_to_transsipment_costs_df['Cost'].values.reshape(15, -1)
transsipment_to_refinement_costs = transsipment_to_refinement_costs_df['Cost'].values.reshape(2, -1)

In [1526]:
direct_shipping_costs

array([[4.25273277, 4.56772552, 4.69648446, 2.67874096, 4.27245069],
       [2.95047222, 2.63661931, 2.85346871, 5.48002613, 3.03749539],
       [4.02568313, 5.40779944, 3.38094894, 5.44123713, 4.40886217],
       [3.46114107, 5.34694389, 4.99366311, 5.96818684, 4.83600407],
       [5.94681429, 4.74917249, 3.72614981, 3.58915858, 4.93741459],
       [3.46939366, 2.75501443, 5.60123055, 4.49910112, 4.22405746],
       [4.7290315 , 4.30193766, 3.67740309, 4.15881121, 4.15956806],
       [5.74380976, 5.57158666, 5.09534814, 5.34986497, 2.62562966],
       [4.25622982, 5.56865574, 4.13714572, 4.90410382, 3.05688139],
       [4.29664067, 5.75680178, 4.87474585, 4.10968275, 4.66040916],
       [4.02164417, 5.38745866, 5.4089004 , 5.18510303, 3.47259321],
       [3.28957805, 3.58831517, 3.62182771, 4.84919596, 4.89636402],
       [2.9072415 , 4.15142098, 4.60651418, 5.22233913, 3.76568815],
       [5.80493436, 3.58642269, 4.32677984, 5.63769746, 3.39212168],
       [5.87454415, 3.572913  , 3.

In [1527]:
shipping_to_transsipment_costs

array([[2.37882586, 0.86384215],
       [1.66698156, 2.11948817],
       [2.17488041, 0.94818376],
       [2.67741553, 0.5924201 ],
       [2.82024783, 0.97232892],
       [2.85389005, 2.6943427 ],
       [1.40702817, 1.42832532],
       [2.48499907, 0.95347666],
       [1.78637201, 0.63669962],
       [0.68852771, 2.25970328],
       [2.40128579, 1.80010665],
       [0.56435019, 2.12484244],
       [1.98606958, 2.34717953],
       [1.63996858, 1.61928103],
       [0.64904663, 1.96881166]])

In [1528]:
transsipment_to_refinement_costs

array([[1.57232851, 3.46547396, 2.24406225, 3.77383915, 3.2626518 ],
       [1.90042452, 1.6131232 , 1.70723241, 2.70539568, 2.25742842]])

In [1529]:
# Create the a single class of decision variables where
# From = {Ca,US,M,C,F} and To = {R1,R2,R3,R4,R5}.
x = model.addVars(25, 5, lb=0, vtype=GRB.CONTINUOUS, name="Direct Shipping")
# From = {I,U,G} and To = {Italy,Greece}.
y = model.addVars(15, 2, lb=0, vtype=GRB.CONTINUOUS, name="Ship to Transshipment")
# From = {Italy,Greece} and To = {R1,R2,R3,R4,R5}.
z = model.addVars(2, 5, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment to Refinement")

In [1530]:
# The objective function
direct_objective = gb.quicksum(direct_shipping_costs[i][j]*x[i,j] for i in range(25) for j in range(5))
trans_objective = gb.quicksum(shipping_to_transsipment_costs[i][j]*y[i,j] for i in range(15) for j in range(2))
trans_to_refinement_objective = gb.quicksum(transsipment_to_refinement_costs[i][j]*z[i,j] for i in range(2) for j in range(5))
model.setObjective(direct_objective + trans_objective + trans_to_refinement_objective, GRB.MINIMIZE)

In [1531]:
# Read capacity and demand from CSV files
direct_shipping_supply_capacity_df = pd.read_csv(r"C:\Users\gabri\Downloads\Capacity_for_Direct_Production_Facilities.csv")
transshipment_supply_capacity_df = pd.read_csv(r"C:\Users\gabri\Downloads\Capacity_for_Transship_Distribution_Centers.csv")
shipping_to_transshipment_supply_capacity_df = pd.read_csv(r"C:\Users\gabri\Downloads\Capacity_for_Transship_Production_Facilities.csv")
refinement_demand_df = pd.read_csv(r"C:\Users\gabri\Downloads\Refinement_Demand.csv")

In [1532]:
# Extract the "Capacity" and "Demand" column
direct_shipping_supply_capacity = direct_shipping_supply_capacity_df['Capacity'].values.reshape(25, -1)
transshipment_supply_capacity = transshipment_supply_capacity_df['Capacity'].values.reshape(2, -1)
shipping_to_transshipment_supply_capacity = shipping_to_transshipment_supply_capacity_df['Capacity'].values.reshape(15, -1)
refinement_demand = refinement_demand_df['Demand'].values.reshape(5, -1)

In [1533]:
direct_shipping_supply_capacity

array([[462],
       [103],
       [460],
       [325],
       [227],
       [217],
       [205],
       [521],
       [548],
       [191],
       [361],
       [411],
       [104],
       [155],
       [285],
       [109],
       [422],
       [438],
       [501],
       [139],
       [462],
       [504],
       [106],
       [132],
       [298]], dtype=int64)

In [1534]:
transshipment_supply_capacity

array([[1317],
       [1453]], dtype=int64)

In [1535]:
shipping_to_transshipment_supply_capacity

array([[374],
       [444],
       [395],
       [245],
       [378],
       [408],
       [435],
       [175],
       [415],
       [503],
       [184],
       [297],
       [450],
       [169],
       [365]], dtype=int64)

In [1536]:
refinement_demand

array([[1537],
       [1748],
       [1940],
       [1838],
       [1665]], dtype=int64)

In [1537]:
# Add the supply constraints from source nodes for direct shipping
for i in range(len(direct_shipping_supply_capacity)):
    model.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")

In [1538]:
# Add the supply constraints from source nodes for transshipment shipping
for i in range(len(shipping_to_transshipment_supply_capacity)):
    model.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")

In [1539]:
# Add the supply constraints from transshipment nodes
model.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
model.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")

<gurobi.Constr *Awaiting Model Update*>

In [1540]:
# Add the flow balance constrainits
model.addConstr(gb.quicksum(y[i,0] for i in range(15)) == gb.quicksum(z[0,k] for k in range(5)), name="Flow Balance 1")
model.addConstr(gb.quicksum(y[i,1] for i in range(15)) == gb.quicksum(z[1,k] for k in range(5)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [1541]:
# Add the demand constraints
for k in range(len(refinement_demand)):  # Iterate over refinement nodes R1 to R5
    model.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")

In [1542]:
# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 49 rows, 165 columns and 360 nonzeros
Model fingerprint: 0x7f8fadb4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 360 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   8.266000e+03   0.000000e+00      0s
      36    2.4188585e+04   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.418858517e+04


In [1543]:
# The status of the model (Optimization Status Codes)
print("Model Status: ", model.status)

Model Status:  2


In [1544]:
# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

Number of Decision Variables:  165


The answer to a)

In [1545]:
# Value of the objective function
print("Total Transportation cost: ", round(model.objVal, 2))

Total Transportation cost:  24188.59


In [1546]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
Direct Shipping[0,3]          462 
Direct Shipping[1,1]          103 
Direct Shipping[2,2]          460 
Direct Shipping[4,3]           86 
Direct Shipping[5,1]          217 
Direct Shipping[7,4]          521 
Direct Shipping[8,4]          548 
Direct Shipping[10,4]          354 
Direct Shipping[11,0]            7 
Direct Shipping[11,2]          404 
Direct Shipping[12,0]          104 
Direct Shipping[13,4]          155 
Direct Shipping[14,3]          285 
Direct Shipping[15,0]          109 
Direct Shipping[17,1]          351 
Direct Shipping[17,4]           87 
Direct Shipping[18,3]          501 
Direct Shipping[20,2]          462 
Direct Shipping[21,3]          504 
Direct Shipping[22,1]          106 
Direct Shipping[23,2]          132 
Ship to Transshipment[0,1]          374 
Ship to Transshipment[2,1]          395 
Ship to Transshipment[3,1]          245 
Ship to Transshipment[6,0]          152 
Ship to Transshipment[7,1]       

In [1547]:
# Get the optimal values of decision variables
to_transshipment_values = model.getAttr('x', y)
to_transshipment_values

{(0, 0): 0.0,
 (0, 1): 374.0,
 (1, 0): 0.0,
 (1, 1): 0.0,
 (2, 0): 0.0,
 (2, 1): 395.0,
 (3, 0): 0.0,
 (3, 1): 245.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (5, 0): 0.0,
 (5, 1): 0.0,
 (6, 0): 152.0,
 (6, 1): 0.0,
 (7, 0): 0.0,
 (7, 1): 24.0,
 (8, 0): 0.0,
 (8, 1): 415.0,
 (9, 0): 503.0,
 (9, 1): 0.0,
 (10, 0): 0.0,
 (10, 1): 0.0,
 (11, 0): 297.0,
 (11, 1): 0.0,
 (12, 0): 0.0,
 (12, 1): 0.0,
 (13, 0): 0.0,
 (13, 1): 0.0,
 (14, 0): 365.0,
 (14, 1): 0.0}

In [1548]:
values_array = np.array(list(to_transshipment_values.values()))

In [1549]:
# Calculate the mean
mean_value = np.mean(values_array)
mean_value

92.33333333333333

In [1550]:
# Get the optimal values of decision variables
transshipment_values = model.getAttr('x', z)
transshipment_values

{(0, 0): 1317.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 971.0,
 (1, 2): 482.0,
 (1, 3): 0.0,
 (1, 4): 0.0}

In [1551]:
# Calculate the total amount of canola oil transshipped
total_transshipped = sum(transshipment_values[i, j] for i in range(2) for j in range(5))
total_transshipped

2770.0

In [1552]:
# Get the optimal values of decision variables
direct_shipping_values = model.getAttr('x', x)
direct_shipping_values

{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 462.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 103.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 460.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 86.0,
 (4, 4): 0.0,
 (5, 0): 0.0,
 (5, 1): 217.0,
 (5, 2): 0.0,
 (5, 3): 0.0,
 (5, 4): 0.0,
 (6, 0): 0.0,
 (6, 1): 0.0,
 (6, 2): 0.0,
 (6, 3): 0.0,
 (6, 4): 0.0,
 (7, 0): 0.0,
 (7, 1): 0.0,
 (7, 2): 0.0,
 (7, 3): 0.0,
 (7, 4): 521.0,
 (8, 0): 0.0,
 (8, 1): 0.0,
 (8, 2): 0.0,
 (8, 3): 0.0,
 (8, 4): 548.0,
 (9, 0): 0.0,
 (9, 1): 0.0,
 (9, 2): 0.0,
 (9, 3): 0.0,
 (9, 4): 0.0,
 (10, 0): 0.0,
 (10, 1): 0.0,
 (10, 2): 0.0,
 (10, 3): 0.0,
 (10, 4): 354.0,
 (11, 0): 7.0,
 (11, 1): 0.0,
 (11, 2): 404.0,
 (11, 3): 0.0,
 (11, 4): 0.0,
 (12, 0): 104.0,
 (12, 1): 0.0,
 (12, 2): 0.0,
 (12, 3): 0.0,
 (12, 4): 0.0,
 (13, 0): 0.0,
 (13, 1): 0.0,
 (13, 2): 0.0,
 (13, 3): 0

In [1553]:
# Calculate the total amount of canola oil directly shipped
direct_shipping = sum(direct_shipping_values[i, j] for i in range(25) for j in range(5))
direct_shipping

5958.0

The answer to b)

In [1554]:
# Calculate the proportion
total_canola_oil = direct_shipping + total_transshipped
proportion_transshipped = total_transshipped / total_canola_oil
proportion_transshipped

0.31736938588450964

In [1555]:
# Create the optimization model
model_modified = gb.Model("Question 1: Canola Oil Objective Function Change")

In [1556]:
# Create the a single class of decision variables where
# From = {Ca,US,M,C,F} and To = {R1,R2,R3,R4,R5}.
x = model_modified.addVars(25, 5, lb=0, vtype=GRB.CONTINUOUS, name="Direct Shipping")
# From = {I,U,G} and To = {Italy,Greece}.
y = model_modified.addVars(15, 2, lb=0, vtype=GRB.CONTINUOUS, name="Ship to Transshipment")
# From = {Italy,Greece} and To = {R1,R2,R3,R4,R5}.
z = model_modified.addVars(2, 5, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment to Refinement")

In [1557]:
# The objective function
direct_objective = gb.quicksum(direct_shipping_costs[i][j]*x[i,j] for i in range(25) for j in range(5))
trans_objective = gb.quicksum(shipping_to_transsipment_costs[i][j]*y[i,j] for i in range(15) for j in range(2))
trans_to_refinement_objective = gb.quicksum(transsipment_to_refinement_costs[i][j]*z[i,j] for i in range(2) for j in range(5))

y_limit = np.sqrt(92)  

# Penalty term for exceeding the limit
penalty = gb.quicksum((y[i,j] - y_limit)**2 for i in range(15) for j in range(2))

# Modified trans_objective with penalty
trans_objective_with_penalty = trans_objective + penalty

# Adding the modified objective to the model
model_modified.setObjective(direct_objective + trans_objective_with_penalty + trans_to_refinement_objective, GRB.MINIMIZE)

In [1558]:
# Add the supply constraints from source nodes for direct shipping
for i in range(len(direct_shipping_supply_capacity)):
    model_modified.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")

In [1559]:
# Add the supply constraints from source nodes for transshipment shipping
for i in range(len(shipping_to_transshipment_supply_capacity)):
    model_modified.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")

In [1560]:
# Add the supply constraints from transshipment nodes
model_modified.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
model_modified.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")

<gurobi.Constr *Awaiting Model Update*>

In [1561]:
# Add the flow balance constrainits
model_modified.addConstr(gb.quicksum(y[i,0] for i in range(15)) == gb.quicksum(z[0,k] for k in range(5)), name="Flow Balance 1")
model_modified.addConstr(gb.quicksum(y[i,1] for i in range(15)) == gb.quicksum(z[1,k] for k in range(5)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [1562]:
# Add the demand constraints
for k in range(len(refinement_demand)):  # Iterate over refinement nodes R1 to R5
    model_modified.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")

In [1563]:
# Optimally solve the problem
model_modified.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 49 rows, 165 columns and 360 nonzeros
Model fingerprint: 0x6ba436d1
Model has 30 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 360 nonzeros
Presolved model has 30 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.970e+02
 Factor NZ  : 7.150e+02
 Factor Ops : 1.391e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.26989301e+08

In [1564]:
# Value of the objective function
print("Total Transportation cost: ", round(model_modified.objVal, 2))

Total Transportation cost:  47023.46


In [1565]:
# Get the optimal values of decision variables
transshipment_values_modified = model_modified.getAttr('x', y)
transshipment_values_modified

{(0, 0): 34.325541279866485,
 (0, 1): 35.2367554139976,
 (1, 0): 34.6814634288198,
 (1, 1): 34.60893240076593,
 (2, 0): 34.42751400293915,
 (2, 1): 35.19458461008363,
 (3, 0): 34.17624644510892,
 (3, 1): 35.372466439918796,
 (4, 0): 34.10483029438945,
 (4, 1): 35.18251202854432,
 (5, 0): 34.08800918734007,
 (5, 1): 34.32150513612831,
 (6, 0): 34.81144012514729,
 (6, 1): 34.954513830189974,
 (7, 0): 34.272454675517714,
 (7, 1): 35.19193815906755,
 (8, 0): 34.62176820390596,
 (8, 1): 35.350326677223634,
 (9, 0): 35.17069035633964,
 (9, 1): 34.53882484973999,
 (10, 0): 34.314311314534756,
 (10, 1): 34.768623165001515,
 (11, 0): 35.23277911330116,
 (11, 1): 34.60625526995009,
 (12, 0): 34.52191942019273,
 (12, 1): 34.49508672363759,
 (13, 0): 34.69496992233712,
 (13, 1): 34.85903597422256,
 (14, 0): 35.19043089648857,
 (14, 1): 34.68427065646435}

In [1566]:
# Calculate the total amount of canola oil transshipped
total_transshipped_modified = sum(transshipment_values_modified[i, j] for i in range(15) for j in range(2))
total_transshipped_modified

1042.0000000011644

Part d)

In [1567]:
# Create the optimization model
model_constraint = gb.Model("Question 1: Canola Oil Constraint Change")

In [1568]:
# Create the a single class of decision variables where
# From = {Ca,US,M,C,F} and To = {R1,R2,R3,R4,R5}.
x = model_constraint.addVars(25, 5, lb=0, vtype=GRB.CONTINUOUS, name="Direct Shipping")
# From = {I,U,G} and To = {Italy,Greece}.
y = model_constraint.addVars(15, 2, lb=0, vtype=GRB.CONTINUOUS, name="Ship to Transshipment")
# From = {Italy,Greece} and To = {R1,R2,R3,R4,R5}.
z = model_constraint.addVars(2, 5, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment to Refinement")

In [1569]:
# The objective function
direct_objective = gb.quicksum(direct_shipping_costs[i][j]*x[i,j] for i in range(25) for j in range(5))
trans_objective = gb.quicksum(shipping_to_transsipment_costs[i][j]*y[i,j] for i in range(15) for j in range(2))
trans_to_refinement_objective = gb.quicksum(transsipment_to_refinement_costs[i][j]*z[i,j] for i in range(2) for j in range(5))
model_constraint.setObjective(direct_objective + trans_objective + trans_to_refinement_objective, GRB.MINIMIZE)

In [1570]:
# Add the supply constraints from source nodes for direct shipping
for i in range(len(direct_shipping_supply_capacity)):
    model_constraint.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")

In [1571]:
# Add the supply constraints from source nodes for transshipment shipping
for i in range(len(shipping_to_transshipment_supply_capacity)):
    model_constraint.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")

In [1572]:
# Add the supply constraints from transshipment nodes
model_constraint.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
model_constraint.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")

<gurobi.Constr *Awaiting Model Update*>

In [1573]:
# Add the flow balance constrainits
model_constraint.addConstr(gb.quicksum(y[i,0] for i in range(15)) == gb.quicksum(z[0,k] for k in range(5)), name="Flow Balance 1")
model_constraint.addConstr(gb.quicksum(y[i,1] for i in range(15)) == gb.quicksum(z[1,k] for k in range(5)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [1574]:
# Add the demand constraints
for k in range(len(refinement_demand)):  # Iterate over refinement nodes R1 to R5
    model_constraint.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")

Added constraint

In [1575]:
# Ratio constraint
model_constraint.addConstr((0.25*(gb.quicksum(x[i,j] for i in range(25) for j in range(5)) + gb.quicksum(z[i,j] for i in range(2) for j in range(5)))) >= gb.quicksum(y[i,j] for i in range(15) for j in range(2)), name="Ratio constraint")

<gurobi.Constr *Awaiting Model Update*>

In [1576]:
# Optimally solve the problem
model_constraint.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 50 rows, 165 columns and 525 nonzeros
Model fingerprint: 0x5628116b
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 50 rows, 165 columns, 525 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   8.266000e+03   0.000000e+00      0s
      35    2.4720228e+04   0.000000e+00   0.000000e+00      0s

Solved in 35 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.472022754e+04


In [1577]:
# Value of the objective function
print("Total Transportation cost: ", round(model_constraint.objVal, 2))

Total Transportation cost:  24720.23


In [1578]:
# Get the optimal values of decision variables
transshipment_values = model_constraint.getAttr('x', z)
transshipment_values

{(0, 0): 1148.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 964.0,
 (1, 2): 70.0,
 (1, 3): 0.0,
 (1, 4): 0.0}

In [1579]:
# Calculate the total amount of canola oil transshipped
total_transshipped = sum(transshipment_values[i, j] for i in range(2) for j in range(5))
total_transshipped

2182.0

In [1580]:
# Get the optimal values of decision variables
direct_shipping_values = model_constraint.getAttr('x', x)
direct_shipping_values

{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 462.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 103.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 460.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 149.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 227.0,
 (4, 3): 0.0,
 (4, 4): 0.0,
 (5, 0): 0.0,
 (5, 1): 217.0,
 (5, 2): 0.0,
 (5, 3): 0.0,
 (5, 4): 0.0,
 (6, 0): 0.0,
 (6, 1): 0.0,
 (6, 2): 205.0,
 (6, 3): 0.0,
 (6, 4): 0.0,
 (7, 0): 0.0,
 (7, 1): 0.0,
 (7, 2): 0.0,
 (7, 3): 0.0,
 (7, 4): 521.0,
 (8, 0): 0.0,
 (8, 1): 0.0,
 (8, 2): 0.0,
 (8, 3): 0.0,
 (8, 4): 548.0,
 (9, 0): 0.0,
 (9, 1): 0.0,
 (9, 2): 0.0,
 (9, 3): 0.0,
 (9, 4): 0.0,
 (10, 0): 0.0,
 (10, 1): 0.0,
 (10, 2): 0.0,
 (10, 3): 0.0,
 (10, 4): 361.0,
 (11, 0): 27.0,
 (11, 1): 0.0,
 (11, 2): 384.0,
 (11, 3): 0.0,
 (11, 4): 0.0,
 (12, 0): 104.0,
 (12, 1): 0.0,
 (12, 2): 0.0,
 (12, 3): 0.0,
 (12, 4): 0.0,
 (13, 0): 0.0,
 (13, 1): 0.0,
 (13, 2): 0.0,
 (13,

In [1581]:
# Calculate the total amount of canola oil directly shipped
direct_shipping = sum(direct_shipping_values[i, j] for i in range(25) for j in range(5))
direct_shipping

6546.0

In [1582]:
# Calculate the proportion
total_canola_oil = direct_shipping + total_transshipped
proportion_transshipped = total_transshipped / total_canola_oil
proportion_transshipped

0.25

e) Choose the constraint route for obvious reasons

# Question 2

In [1583]:
# Create the optimization model
question_2_model = gb.Model("Question 2: Sunnyshore Bay")

In [1584]:
# Create the three classes of decision variables where each Python
# variable represents a different number of Gurobi decision variables
B = question_2_model.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Borrow")
w = question_2_model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Wealth")

In [1585]:
# The objective function
question_2_model.setObjective(w[3], GRB.MAXIMIZE)

In [1586]:
# Add the balance constraints
question_2_model.addConstr(w[0] == 140000 + 180000 - 300000 + B[0] + B[1] + B[2], "May Balance Constraint")
question_2_model.addConstr(w[1] == w[0] + 260000 - 400000 + B[3] + B[4] - 1.0175*B[2], "June Balance Constraint")
question_2_model.addConstr(w[2] == w[1] + 420000 - 350000 + B[5] - 1.0225*B[1] - 1.0175*B[4], "July Balance Constraint")
question_2_model.addConstr(w[3] == w[2] + 580000 - 200000 - 1.0275*B[0] - 1.0225*B[3] - 1.0175*B[5], "August Balance Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [1587]:
# Add the cash flow constraints
May_Cash_Flow_Constraint = question_2_model.addConstr(w[0] >= 25000, "May Cash Flow Constraint")
June_Cash_Flow_Constraint = question_2_model.addConstr(w[1] >= 20000, "June Cash Flow Constraint")
July_Cash_Flow_Constraint = question_2_model.addConstr(w[2] >= 35000, "July Cash Flow Constraint")
August_Cash_Flow_Constraint = question_2_model.addConstr(w[3] >= 18000, "August Cash Flow Constraint")

In [1588]:
# Add the borrowing constraints
question_2_model.addConstr((B[0] + B[1] + B[2]) <= 250000, "May Borrowing Constraint")
question_2_model.addConstr((B[3] + B[4]) <= 150000, "June Borrowing Constraint")
question_2_model.addConstr(B[5] <= 350000, "July Borrowing Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [1589]:
# Ratio constraint
question_2_model.addConstr((0.65*(w[0] + w[1])) <= w[2], name="Ratio constraint")

<gurobi.Constr *Awaiting Model Update*>

In [1590]:
# Optimally solve the problem
question_2_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 12 rows, 10 columns and 32 nonzeros
Model fingerprint: 0xcc017ff0
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 5 rows and 0 columns
Presolve time: 0.00s
Presolved: 7 rows, 10 columns, 27 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   1.000000e+30   8.000000e+00      0s
       6    3.2709527e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.270952703e+05


In [1591]:
# The status of the model (Optimization Status Codes)
print("Model Status: ", question_2_model.status)

Model Status:  2


In [1592]:
# Number of variables in the model
print("Number of Decision Variables: ", question_2_model.numVars)

Number of Decision Variables:  10


In [1593]:
# Value of the objective function
print("Total Amount of Money: ", round(question_2_model.objVal, 2))

Total Amount of Money:  327095.27


In [1594]:
# Print the decision variables
print(question_2_model.printAttr('X'))


    Variable            X 
-------------------------
   Borrow[0]         5000 
   Borrow[3]      80945.9 
   Borrow[4]      54054.1 
   Wealth[0]        25000 
   Wealth[1]        20000 
   Wealth[2]        35000 
   Wealth[3]       327095 
None


a) The answer is 6

b) June cash balance >= $20,000

c) The formula: w[2] / (w[0] + w[1]) <= 0.65 or 65%
Written out, July cash balance / (May cash balance + June cash balance) <= 0.65 or 65%

To make it linear: 0.65*(w[0] + w[1]) <= w[2]
Written out, 0.65*(May cash balance + June cash balance) <= July cash balance

d)

In [1595]:
# Get the optimal values of decision variables
borrowing_money = question_2_model.getAttr('x', B)

# Display the optimal values
print("Optimal Borrowing Values:")
for i in range(len(borrowing_money)):
    print(f"Borrow[{i}] = {borrowing_money[i]}")

# Calculate the total amount borrowed
total_borrowing_amount = borrowing_money[0] + borrowing_money[1] + borrowing_money[2] + borrowing_money[3] + borrowing_money[4] + borrowing_money[5]
print("\nTotal Borrowing Amount:", total_borrowing_amount)
total_repay_amount = 1.0275*borrowing_money[0] + 1.0225*borrowing_money[1] + 1.0175*borrowing_money[2] + 1.0225*borrowing_money[3] + 1.0175*borrowing_money[4] + 1.0175*borrowing_money[5]
print("\nTotal Repayment Amount:", total_repay_amount)


Optimal Borrowing Values:
Borrow[0] = 5000.0
Borrow[1] = 0.0
Borrow[2] = 0.0
Borrow[3] = 80945.94594594595
Borrow[4] = 54054.05405405405
Borrow[5] = 0.0

Total Borrowing Amount: 140000.0

Total Repayment Amount: 142904.72972972973


e) Looking above, the answer is $5,000

f) $327,095.27

g)

In [1596]:
# Print sensitivity information
print("")
print(f"Sensitivity Information for June Cash Flow Constraint {June_Cash_Flow_Constraint.pi:.2f}:")
print("(LHS, RHS, Slack): ", (question_2_model.getRow(June_Cash_Flow_Constraint).getValue(), June_Cash_Flow_Constraint.RHS, June_Cash_Flow_Constraint.slack))
print("Shadow Price: ", June_Cash_Flow_Constraint.pi)
print("Range of Feasibility: ", (June_Cash_Flow_Constraint.SARHSUp, June_Cash_Flow_Constraint.SARHSLow))



Sensitivity Information for June Cash Flow Constraint -0.02:
(LHS, RHS, Slack):  (20000.0, 20000.0, 0.0)
Shadow Price:  -0.01758599508599512
Range of Feasibility:  (28846.153846153844, -0.0)


In [1597]:
# Check if the optimization was successful
if question_2_model.status == gb.GRB.OPTIMAL:
    # Print the sensitivity analysis for the amount sold
    print("Optimal Amount Sold:")
    print(f"{'1'} = {B[0].x, B[0].RC, B[0].SAObjUp, B[0].SAObjLow}")
    print(f"{'2'} = {B[1].x, B[1].RC, B[1].SAObjUp, B[1].SAObjLow}")
    print(f"{'3'} = {B[2].x, B[2].RC, B[2].SAObjUp, B[2].SAObjLow}")
    print(f"{'4'} = {B[3].x, B[3].RC, B[3].SAObjUp, B[3].SAObjLow}")
    print(f"{'5'} = {B[4].x, B[4].RC, B[4].SAObjUp, B[4].SAObjLow}")
    print(f"{'6'} = {B[5].x, B[5].RC, B[5].SAObjUp, B[5].SAObjLow}")
else:
    print("Optimization was not successful.")

Optimal Amount Sold:
1 = (5000.0, 0.0, 0.0050000000000001155, -2.457002456979218e-05)
2 = (0.0, -2.457002456979218e-05, 2.457002456979218e-05, -inf)
3 = (0.0, -0.012893749999999926, 0.012893749999999926, -inf)
4 = (80945.94594594595, 0.0, 2.4449877750379998e-05, -0.0050000000000001155)
5 = (54054.05405405405, 0.0, 0.012806250000000146, -2.4449877750379998e-05)
6 = (0.0, -0.012585995085995227, 0.012585995085995227, -inf)


In [1598]:
# Change in Objective Function Value
change_in_ofv = June_Cash_Flow_Constraint.pi * (27500 - June_Cash_Flow_Constraint.RHS)

# Additional money needed to be borrowed
additional_borrowing = -change_in_ofv  # Note: Multiply by -1 to make it positive

print("Change in Objective Function Value:", change_in_ofv)
print("Additional Money Needed to be Borrowed:", additional_borrowing)

Change in Objective Function Value: -131.8949631449634
Additional Money Needed to be Borrowed: 131.8949631449634


In [1599]:
# Create the optimization model
question_2_model2 = gb.Model("Question 2: Sunnyshore Bay")

# Create the three classes of decision variables where each Python
# variable represents a different number of Gurobi decision variables
B = question_2_model2.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Borrow")
w = question_2_model2.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Wealth")

# The objective function
question_2_model2.setObjective(w[3], GRB.MAXIMIZE)

# Add the balance constraints
question_2_model2.addConstr(w[0] == 140000 + 180000 - 300000 + B[0] + B[1] + B[2], "May Balance Constraint")
question_2_model2.addConstr(w[1] == w[0] + 260000 - 400000 + B[3] + B[4] - 1.0175*B[2], "June Balance Constraint")
question_2_model2.addConstr(w[2] == w[1] + 420000 - 350000 + B[5] - 1.0225*B[1] - 1.0175*B[4], "July Balance Constraint")
question_2_model2.addConstr(w[3] == w[2] + 580000 - 200000 - 1.0275*B[0] - 1.0225*B[3] - 1.0175*B[5], "August Balance Constraint")

# Add the cash flow constraints
May_Cash_Flow_Constraint = question_2_model2.addConstr(w[0] >= 25000, "May Cash Flow Constraint")
June_Cash_Flow_Constraint = question_2_model2.addConstr(w[1] >= 27500, "June Cash Flow Constraint")
July_Cash_Flow_Constraint = question_2_model2.addConstr(w[2] >= 35000, "July Cash Flow Constraint")
August_Cash_Flow_Constraint = question_2_model2.addConstr(w[3] >= 18000, "August Cash Flow Constraint")

# Add the borrowing constraints
question_2_model2.addConstr((B[0] + B[1] + B[2]) <= 250000, "May Borrowing Constraint")
question_2_model2.addConstr((B[3] + B[4]) <= 150000, "June Borrowing Constraint")
question_2_model2.addConstr(B[5] <= 350000, "July Borrowing Constraint")

# Ratio constraint
question_2_model2.addConstr((0.65*(w[0] + w[1])) <= w[2], name="Ratio constraint")

# Optimally solve the problem
question_2_model2.optimize()

# Value of the objective function
print("Total Amount of Money: ", round(question_2_model2.objVal, 2))


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 12 rows, 10 columns and 32 nonzeros
Model fingerprint: 0x502c54de
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 5 rows and 0 columns
Presolve time: 0.00s
Presolved: 7 rows, 10 columns, 27 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   1.000000e+30   8.000000e+00      0s
       6    3.2696338e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.269633753e+05
Total Amount of Money:  326963.38


In [1600]:
# Get the optimal values of decision variables
borrowing_money = question_2_model2.getAttr('x', B)

# Display the optimal values
print("Optimal Borrowing Values:")
for i in range(len(borrowing_money)):
    print(f"Borrow[{i}] = {borrowing_money[i]}")

# Calculate the total amount borrowed
total_borrowing_amount = borrowing_money[0] + borrowing_money[1] + borrowing_money[2] + borrowing_money[3] + borrowing_money[4] + borrowing_money[5]
print("\nTotal Borrowing Amount:", total_borrowing_amount)
total_repay_amount = 1.0275*borrowing_money[0] + 1.0225*borrowing_money[1] + 1.0175*borrowing_money[2] + 1.0225*borrowing_money[3] + 1.0175*borrowing_money[4] + 1.0175*borrowing_money[5]
print("\nTotal Repayment Amount:", total_repay_amount)

Optimal Borrowing Values:
Borrow[0] = 5000.0
Borrow[1] = 0.0
Borrow[2] = 0.0
Borrow[3] = 81074.93857493858
Borrow[4] = 61425.06142506142
Borrow[5] = 0.0

Total Borrowing Amount: 147500.0

Total Repayment Amount: 150536.6246928747


h)